# Data Cleaning

### Import
- Data is given with NaN Values when rows are repeated, so fill these with previous value. Pandas method = 'ffill'.
- Convert Frequency values to floats.
- Convert dates to datetime

In [1]:
import pandas as pd

df = pd.read_csv('rl_data.csv', dtype='object')

# Fill NaN wih previous value for "Provincia_FREQ" and "Municipio"
df['Provincia_FREQ'] = df['Provincia_FREQ'].fillna(method='ffill')
df['Municipio'] = df['Municipio'].fillna(method='ffill')

# "Frecuencia" to float
df['Frecuencia'] = df['Frecuencia'].str.replace(",",".")
df['Frecuencia'] = df['Frecuencia'].astype('float')

#Put all frequencies in MhZ
df.loc[ df['Unit'] == 'GHz', 'Frecuencia'] = df['Frecuencia']*1000
df['Unit']= 'MHz'

# "Dates" to DateTime
df['FConcesion'] = pd.to_datetime(df['FConcesion'], format="%d/%m/%Y", errors='coerce')
df['FCaducidad'] = pd.to_datetime(df['FCaducidad'], format="%d/%m/%Y", errors='coerce')

df.head(3)

,Comunidad,Ref,Titular,NIF/CIF,DomicilioSocial,Localidad,Provincia_RL,CP,FConcesion,FCaducidad,SusceptibleConcesion,SusceptibleMutualizacion,Transferencia,Provincia_FREQ,Municipio,Frecuencia,Unit
0,PAÍS VASCO,BIBI-0700268,"ITELAZPI,S.A.",A95282216,"CL TEKNOLOGI ELKARTEGIA, EDIFICIO 101",ZAMUDIO,BIZKAIA,48170,2007-11-07,2027-12-31,NaN,NaN,NaN,ARABA/ÁLAVA,AMURRIO,22074.5,MHz
1,PAÍS VASCO,BIBI-0700268,"ITELAZPI,S.A.",A95282216,"CL TEKNOLOGI ELKARTEGIA, EDIFICIO 101",ZAMUDIO,BIZKAIA,48170,2007-11-07,2027-12-31,NaN,NaN,NaN,BIZKAIA,ABADIÑO,12765.0,MHz
2,PAÍS VASCO,BIBI-0700268,"ITELAZPI,S.A.",A95282216,"CL TEKNOLOGI ELKARTEGIA, EDIFICIO 101",ZAMUDIO,BIZKAIA,48170,2007-11-07,2027-12-31,NaN,NaN,NaN,BIZKAIA,ABADIÑO,12793.0,MHz


We still have some NaN dates...

In [2]:
df.FConcesion.isna().sum()

28

### Homogenize Companies's Data
There are 190 different company IDs. Some of the companies' addresses, localities, etc have typos, so we need to homogenize it.

In [3]:
len(df['NIF/CIF'].unique())

190

1. Each feature is grouped by `NIF/CIF` to look for different values with a same ID.
2. A diccionary is created in each case where:
    - Key = 'NIF/ID'
    - Value = The unique value for that ID that will be used for the feature
3. Values are replaced in original DataFrame

In [4]:
# Features to homogenize
cols = ['Titular','DomicilioSocial','Localidad','Provincia_RL','CP']

for col in cols:
    # Group by NIF/CIF and "Feature". Sort by count of ocurrences (descending) to get the most used value of that feature for that ID.
    dfGrouped = df.groupby(['NIF/CIF',col], as_index=False)['Ref'].count().sort_values(by=['NIF/CIF','Ref'], ascending=[True,False])

    uniqueValues = {}
    # For each row. If it's different to the previous: take it. Otherwise, go to the next.
    for i,r in dfGrouped.iterrows():
        if i == 0: 
            uniqueValues[r['NIF/CIF']] = r[col]
            prev = r['NIF/CIF']
        else:
            if r['NIF/CIF'] != prev:
                uniqueValues[r['NIF/CIF']] = r[col]
                prev = r['NIF/CIF']
            else: prev = r['NIF/CIF']
    
    # Replace all values of that Feature for each ID with the unique value chosen.
    df[col] = df['NIF/CIF'].replace(uniqueValues)

### Rename Columns

In [6]:
df.to_csv('data.csv',index=False)